In [4]:
from scipy import stats
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, RandomizedSearchCV, GridSearchCV, RepeatedStratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, recall_score, precision_score, precision_recall_curve, roc_curve
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Chapter 3

In [5]:
mnist = fetch_openml('mnist_784', version=1, cache=True)

In [8]:
# Get Data
X = mnist['data']
y= mnist['target']

1. Build classifier for MNIST dataset that achieves over 97% accuracy on the test set

In [9]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]
strat_split = StratifiedShuffleSplit(n_splits=1, train_size=.5, random_state=42)
train_idx, val_idx = next(strat_split.split(X_train, y_train))
training = X_train[train_idx]
train_targ = y_train[train_idx]
val = X_train[val_idx]
val_targ = y_train[val_idx]

In [10]:
pipeline = Pipeline([('std_scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=4))]) 
pgrid = {'knn__weights': ['uniform', 'distance'], 
         'knn__n_neighbors': [3, 5, 7, 9],
         'knn__leaf_size': [25, 30, 35],
         'knn__p': [1, 2, 3, 4]
        }
stratfold = RepeatedStratifiedKFold(n_splits=3, random_state=42)
grid_cv = GridSearchCV(pipeline, pgrid, cv=stratfold, scoring='accuracy', return_train_score=True)

In [ ]:
grid_cv.fit(training, train_targ)

# Chapter 4

## 1. Which linear regression training algorithm can you use if you have a training set with millions of features? 

SVD/Normal Equations approach scales quadratically with features, so this is not possible to use. Whereas gradient descent algorithms scale linearly with feature growth, particularly SGD and Mini-batch GD. Batch gradient can possibly be used if the data can fit into memory.  

## 2. Suppose the features in your training set have very different scales. Which algorithm might suffer from this, and how? What can you do about it? 

Regularization requires features to be of similar scale, particularly LASSO regression. If not convergence to optimal answer may not occur. This is due to large weights being penalized and therefore the value of the feature will influence whether or not regularization ignores or penalizes. Additionally GD algorithms will converge very slowly, potentially, if the data is on different scales. To fix these issues, one should scale the data (e.g. standardization). 

## 3. Can Gradient Descent get stuck in a local minimum when training a Logistic Regression model? 

No, the logistic regression model cost function is a convex function, which guarantees a global minimum will be reached. 

## 4. Do all GD algorithms lead to the same model, provided you let them run long enough? 
No, SGD and mini-batch gradient descent are not deterministic in nature. They will converge about the global optimum but not settle at a fixed place. That is, this will be the behavior apart from gradually adjusting the learning rate. 

## 5. Suppose you use Batch GD and you plot the validation error at every epoch. If you notice that the validation error consistently goes up, what is likely going on? How can you fix this? 

If the training error is going down, then the model being used is likely over-fitting to the training data. If the training error is also going up or remaining at a steady state, it can be that the learning rate is too high and batch GD is bouncing around. In the first case, one can apply regularization or can simply stop training after the validation error starts to rise. In the second one can choose a smaller learning rate.  


## 6. Is it a good idea to stop MB GD immediately when the validation error goes up? 

Due to the stochastic nature of MB GD, it is better to wait until the validation error goes above a threshold and remains above a threshold for a period of time. That is to make sure the model is still not improving. Then go back to the model configuration when the validation error crossed the threshold initially. 


## 7. Which GD algorithm will reach the vicinity of the optimal solution the fastest? Which will actually converge? How can you make the others converge as well? 

SGD will be fastest. Batch gradient descent will converge. To make SGD and MB GD converge, shrink the learning rate as SGD/MBGD gets closer to the optimum solution. 


## 8. Suppose you are using Polynomial Regression. You plot the learning curves and you notice that the there is a large gap between the training error and the validation error. What is happening? What are 3 ways to solve this?

This is a sign of overfitting. Can apply regularization, use a simpler model, or get more training data


## 9. Suppose you are using Ridge Regression and you notice that the training error and the validation error are almost equal and fairly high. Would you say that the model suffers from high bias or high variance? Should you increase the regularization hyperparameter $\alpha$ or reduce it? 

This indicates an under-fitted model, suffering from high bias, and reducing regularization via reducing the $\alpha$ value may help. 


## 10. Why would you want to use: 

a. Ridge Regression instead of plain Linear Regression?
Applying regularization is a good idea to prevent overfitting so that the model can extend to unseen data. 
b. Lasso instead of Ridge Regression? 
Many of the parameters may not be important and this will perform feature selection. In the absence of this, ridge regression would be preferred.  
c. Elastic Net instead of Lasso? 
If some of the parameters are strongly correlated or the number of features is much greater than the training samples, then Lasso may become unstable.


## 11. Suppose you want to classify pictures as outdoor/indoor and daytime/nighttime. Should you implement two Logistic Regressions or one Softmax Regression classifier? 

Softmax is a multilabel classifier, not a multi-output classifier. There you would need to implement two Logistic Regression classifiers. 